In [1]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Activation, Flatten, Reshape, LSTM, Conv1D, MaxPooling1D, Conv2D, Input, Concatenate, Add, Embedding
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from keras.utils import np_utils
from sklearn.utils import shuffle

import keras
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers import BatchNormalization
from keras.layers import ELU, PReLU, LeakyReLU
#from keras.layers.advanced_activations import ELU
from keras.initializers import glorot_uniform
from keras import backend as K
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
#from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import save_model


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.constraints import maxnorm
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
import h5py
import numpy as np
import os,random
from tensorflow.keras.layers import Input,Reshape,ZeroPadding2D,Conv2D,Dropout,Flatten,Dense,Activation,MaxPooling2D,AlphaDropout
from tensorflow.keras import layers
import tensorflow.keras.models as Model
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa

In [3]:
NB_CLASSES = 4 # number of outputs = number of classes
VERBOSE = 1
BATCH_SIZE = 10
NB_EPOCH = 30
LENGTH = 500
target_names = ['Non-MO', 'MO']
file_short = "D:/BMO/Main/savedModels/_short.h5"
file_long = "D:/BMO/Main/savedModels/_long.h5"
file_best = "D:/BMO/Main/savedModels/_best.h5"

In [4]:
%%time

print(".........Loading MO (2-state) dataset..........")

# sep separates each column and header = none means it is going to read from row 1
re2psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/BPSK-2/real.xls',sep='\t', header=None) 
im2psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/BPSK-2/im.xls',sep='\t', header=None)

# sep separates each column and header = none means it is going to read from row 1
re4psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/QPSK-2/real.xls',sep='\t', header=None) 
im4psk_2st = pd.read_csv('D:/BMO/Main/bigDataset/QPSK-2/im.xls',sep='\t', header=None)

# sep separates each column and header = none means it is going to read from row 1
re16qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/16-QAM-2/real.xls',sep='\t', header=None) 
im16qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/16-QAM-2/im.xls',sep='\t', header=None)

# sep separates each column and header = none means it is going to read from row 1
re64qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/64-QAM-2/real.xls',sep='\t', header=None) 
im64qam_2st = pd.read_csv('D:/BMO/Main/bigDataset/64-QAM-2/im.xls',sep='\t', header=None)

#merge two dataframes into one
df2psk_2st = re2psk_2st.append(im2psk_2st)
df2psk_2st['Mod'] = 0 # 0 = BPSK 2-state
#merge two dataframes into one
df4psk_2st = re4psk_2st.append(im4psk_2st)
df4psk_2st['Mod'] = 1 # 1 = QPSK 2-state
#merge two dataframes into one
df16qam_2st = re16qam_2st.append(im16qam_2st)
df16qam_2st['Mod'] = 2 # 2 = 16-QAM 2-state
#merge two dataframes into one
df64qam_2st = re64qam_2st.append(im64qam_2st)
df64qam_2st['Mod'] = 3 # 3 = 64-QAM 2-state

# combine all
df_2st = df2psk_2st.append(df4psk_2st)
df_2st = df_2st.append(df16qam_2st)
df_2st = df_2st.append(df64qam_2st)

#df_mo = df_2st
#df_mo = df_mo.sample(frac = 1)
print('df_mo Shape = ', df_2st.shape)

.........Loading MO (2-state) dataset..........
df_mo Shape =  (56000, 2049)
CPU times: total: 35.6 s
Wall time: 35.6 s


In [5]:
print("..........Combining all data..........")

# combine all
#df = df_mo.append(df_nonmo)
df_all = df_2st.sample(frac = 1)
print('df_all Shape = ', df_all.shape)
print("..........Shuffling done..........")

# reducing samples to tune only
df_all = df_all.iloc[:6384]
print('df_all Size = ', df_all.size)
print('Dimension = ', df_all.ndim)
print('Combined Shape = ', df_all.shape)

..........Combining all data..........
df_all Shape =  (56000, 2049)
..........Shuffling done..........
df_all Size =  13080816
Dimension =  2
Combined Shape =  (6384, 2049)


In [6]:
# Separating X and y
y = df_all['Mod'] # 1D targer vector
X = df_all.drop(columns='Mod')

INPUT_SHAPE = (LENGTH,1)
    
X.drop(X.iloc[:, LENGTH:2048], inplace = True, axis = 1)

X = np.expand_dims(X, -1)

# Split into training/testing sets with 20% split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=1) 

print("..........Training set..........")
print(X_train.shape)
print(y_train.shape)
print("..........Validation set..........")
print(X_val.shape)
print(y_val.shape)
print("..........Testing set..........")
print(X_test.shape)
print(y_test.shape)

# Convert class vectors to categorical classes matrices
y_train = np_utils.to_categorical(y_train, NB_CLASSES)
y_test = np_utils.to_categorical(y_test, NB_CLASSES)
y_val = np_utils.to_categorical(y_val, NB_CLASSES)

..........Training set..........
(4596, 500, 1)
(4596,)
..........Validation set..........
(511, 500, 1)
(511,)
..........Testing set..........
(1277, 500, 1)
(1277,)


In [7]:
from keras.constraints import maxnorm

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
   # model.add(Embedding(input_dim = 188, output_dim = 50, input_length = INPUT_SHAPE))
    model.add(LSTM(LENGTH))
    #model.add(Dropout(0.5))
    #model.add(Dense(NB_CLASSES, activation='sigmoid'))
    
    model.add(BatchNormalization(axis=-1))
    model.add(ELU(alpha=1.0, name='activation1'))
    #model.add(MaxPooling1D(pool_size=pool_size[1], strides=pool_stride_size[1],
    #                           padding='same', name='pool1'))
    model.add(Dropout(0))
    
    # Output layer
    model.add(Flatten(name='flatten1'))
    model.add(Dense(NB_CLASSES, kernel_initializer=glorot_uniform(seed=0), name='dense1'))
    model.add(Activation('softmax', name="softmax"))
    
    # Output layer
   # model.add(Flatten(name='flatten1'))
   # model.add(Dense(NB_CLASSES, kernel_initializer=glorot_uniform(seed=0), name='dense1'))
   # model.add(Activation('softmax', name="softmax"))
    
    # Compile model
    model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01, momentum=0.6), metrics=["accuracy"])
    
    return model

In [8]:
%%time
model_lr = create_model()
# simple early stopping
es_lr = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc_lr = ModelCheckpoint(file_best, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
# fit model
model_history = model_lr.fit(X_train, y_train, epochs=NB_EPOCH, verbose=VERBOSE, 
                        validation_data=(X_val, y_val), callbacks=[es_lr, mc_lr])

Epoch 1/30
144/144 [==============================] - ETA: 0s - loss: 1.4076 - accuracy: 0.2493
Epoch 1: val_accuracy improved from -inf to 0.25049, saving model to D:/BMO/Main/savedModels\_best.h5
144/144 [==============================] - 425s 3s/step - loss: 1.4076 - accuracy: 0.2493 - val_loss: 1.3877 - val_accuracy: 0.2505
Epoch 2/30
144/144 [==============================] - ETA: 0s - loss: 1.4025 - accuracy: 0.2574
Epoch 2: val_accuracy improved from 0.25049 to 0.25832, saving model to D:/BMO/Main/savedModels\_best.h5
144/144 [==============================] - 424s 3s/step - loss: 1.4025 - accuracy: 0.2574 - val_loss: 1.3878 - val_accuracy: 0.2583
Epoch 3/30
144/144 [==============================] - ETA: 0s - loss: 1.4022 - accuracy: 0.2517
Epoch 3: val_accuracy improved from 0.25832 to 0.29941, saving model to D:/BMO/Main/savedModels\_best.h5
144/144 [==============================] - 433s 3s/step - loss: 1.4022 - accuracy: 0.2517 - val_loss: 1.3855 - val_accuracy: 0.2994
Epoc

In [11]:
 # Start evaluating model with testing data
score_test = model_lr.evaluate(X_test, y_test, verbose=VERBOSE)
y_pred = model_lr.predict(X_test)
    
y_pred1=np.argmax(y_pred, axis=1)
y_test1=np.argmax(y_test, axis=1)
print(classification_report(y_test1,y_pred1))

40/40 [==============================] - 44s 1s/step
              precision    recall  f1-score   support

           0       0.30      0.29      0.30       342
           1       0.26      0.11      0.15       310
           2       0.29      0.35      0.32       324
           3       0.23      0.33      0.27       301

    accuracy                           0.27      1277
   macro avg       0.27      0.27      0.26      1277
weighted avg       0.27      0.27      0.26      1277

